# Quandl

```bash
export QUANDL_API_KEY="your key"
```

In [ ]:
from zipline.data.bundles import register, ingest
from zipline.data.bundles.quandl import quandl_bundle

# 注册 Quandl bundle
register(
    'quandl',
    quandl_bundle,
    calendar_name='NYSE',
)

# 执行 ingest
ingest('quandl')

# Yahoo-finance

## Step 1. Preparing the data

In [3]:
import yfinance as yf
import pandas as pd

def download_from_yahoo(
    *symbols: str,
    start_date: str | pd.Timestamp,
    end_date: str | pd.Timestamp,
):
    # 从yfinance下载数据
    data = yf.download(
        symbols,
        start=start_date,
        end=end_date,
        group_by='ticker',
    )

    symbol_data: pd.DataFrame = (
        data
        .reset_index()
        .rename(
            columns={
                'Date': 'date',
                'Open': 'open',
                'High': 'high',
                'Low': 'low',
                'Close': 'close',
                'Volume': 'volume',
                # 'Adj Close': 'adj_close'
            },
        )
        .set_index('date')
    )
    return symbol_data

# 下载数据

dji = [
    'AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'DOW', 'GS', 'HD',
    'IBM',
    'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE',
    'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT', 'XOM', 'DJIA', "^DJI",
]
start_date = pd.Timestamp('2000-01-03')
end_date = pd.Timestamp('2025-03-20')

df = download_from_yahoo(*dji, start_date=start_date, end_date=end_date)
df

[*********************100%***********************]  31 of 31 completed


Ticker            NKE                                                   MRK  \
Price            open       high        low      close    volume       open   
date                                                                          
2000-01-03   4.665723   4.689377   4.517886   4.553367   8014400  26.319331   
2000-01-04   4.446924   4.476492   4.293174   4.305001   9810400  25.696095   
2000-01-05   4.328656   4.577021   4.328656   4.553367   6542400  24.953040   
2000-01-06   4.529713   4.529713   4.405531   4.529713   4891200  26.007719   
2000-01-07   4.511973   4.541540   4.446925   4.529713   3993600  26.774777   
...               ...        ...        ...        ...       ...        ...   
2025-03-13  73.180000  73.940002  71.669998  72.639999  10195900  92.600026   
2025-03-14  73.279999  73.540001  71.320000  71.660004  12183000  93.472485   
2025-03-17  72.139999  73.949997  72.029999  73.699997  15996900  93.650002   
2025-03-18  73.889999  74.160004  72.720001  73.309998   8972300  95.230003   
2025-03-19  72.639999  73.430000  72.290001  72.989998  10830500  95.250000   

Ticker                                                 ...         KO  \
Price            high        low      close    volume  ...       open   
date                                                   ...              
2000-01-03  26.319331  25.504341  25.935806   6265782  ...  14.315030   
2000-01-04  26.007707  24.833167  25.024929   7894689  ...  13.913966   
2000-01-05  26.415226  24.953040  26.007730   7963018  ...  13.929380   
2000-01-06  26.535066  25.959780  26.223454   4989004  ...  14.052791   
2000-01-07  28.860189  26.702864  28.740337  10871218  ...  14.284169   
...               ...        ...        ...       ...  ...        ...   
2025-03-13  95.514840  92.600026  93.898804  16570100  ...  69.735383   
2025-03-14  94.067345  92.649592  93.760002  12681100  ...  68.510002   
2025-03-17  95.300003  93.080002  94.790001  12237900  ...  69.239998   
2025-03-18  95.410004  93.839996  94.720001   8670100  ...  70.110001   
2025-03-19  95.430000  93.639999  94.019997  11355900  ...  68.550003   

Ticker                                                       WBA             \
Price            high        low      close    volume       open       high   
date                                                                          
2000-01-03  14.315030  13.636300  13.913962  10997000  16.151226  16.257484   
2000-01-04  14.021946  13.728858  13.929392   7308000  15.761617  16.080391   
2000-01-05  14.176190  13.836826  14.052785   9457400  15.726205  15.832463   
2000-01-06  14.237899  13.960237  14.068216   7129200  15.690778  15.797036   
2000-01-07  14.993750  14.284169  14.993750  11474000  15.336585  15.726198   
...               ...        ...        ...       ...        ...        ...   
2025-03-13  69.735383  68.772487  69.110001  15746600  11.180000  11.320000   
2025-03-14  69.260002  68.379997  69.160004  14205100  11.250000  11.280000   
2025-03-17  70.169998  69.080002  70.120003  15634800  11.220000  11.290000   
2025-03-18  70.190002  69.300003  69.379997  12657600  11.250000  11.270000   
2025-03-19  69.379997  68.540001  69.279999  14780000  11.180000  11.200000   

Ticker                                      
Price             low      close    volume  
date                                        
2000-01-03  15.726194  16.186646   2095900  
2000-01-04  15.372004  15.584520   1709100  
2000-01-05  15.478269  15.761624   2167800  
2000-01-06  15.230326  15.301165   2595400  
2000-01-07  14.946972  15.690779   3629900  
...               ...        ...       ...  
2025-03-13  11.180000  11.260000  17385900  
2025-03-14  11.210000  11.230000  10311400  
2025-03-17  11.170000  11.260000  22417700  
2025-03-18  11.170000  11.170000  10076800  
2025-03-19  11.160000  11.160000   8769900  

[6341 rows x 155 columns]

## Save to local

In [4]:
import tqdm
import os

def save_to_local(df: pd.DataFrame, folder_or_file: str, multiple_tickers: bool = True):
    if multiple_tickers:
        folder = os.path.expanduser(folder_or_file)
        os.makedirs(folder, exist_ok=True)
        for ticker in tqdm.tqdm(df.columns.get_level_values('Ticker').unique()):
            df[ticker].to_csv(f'{folder}/{ticker}.csv')
    else:
        df.to_csv(folder_or_file)

save_to_local(
    df,
    '~/.data/test/daily',  # daily is important for zipline
    multiple_tickers=True
)

100%|██████████| 31/31 [00:00<00:00, 35.56it/s]


For example, `~/.data/test/AAPL.csv`

```txt
date,open,high,low,close,volume
2000-01-03,0.7890165672669442,0.8463821917967449,0.7650355602037866,0.8421505093574524,535796800
2000-01-04,0.8144081442752411,0.8322759248125872,0.7612741972199477,0.7711488604545593,512377600
2000-01-05,0.7805525289242127,0.8318057876126348,0.7749103516967237,0.782433271408081,778321600
2000-01-06,0.7984212724934108,0.8050038469645777,0.7147228717803955,0.7147228717803955,767972800
2000-01-07,0.7260080711321768,0.759863747898158,0.7184851510326266,0.7485784888267517,460734400
2000-01-10,0.7673866806511616,0.7692674233861818,0.7128421781123165,0.73541259765625,505064000
```

## Step 3. Change `~/.zipline/extension.py`

```python
import pandas as pd

from zipline.data.bundles import register
from zipline.data.bundles.csvdir import csvdir_equities


start_session = pd.Timestamp('2000-01-03')
end_session = pd.Timestamp('2025-03-20')


register(
    'us_equities',
    csvdir_equities(
        ['daily'],
        '/home/vscode/.data/test',
    ),
    calendar_name='NYSE',  # US equities
    start_session=start_session,
    end_session=end_session
)
```

## Step 4. Ingest dataset in Terminal

```bash
$ zipline ingest -b us_equities
```

and check

```bash
$ zipline bundles

csvdir <no ingestions>
quandl 2025-03-19 20:04:04.649765
quandl 2025-03-19 20:02:30.584794
quantopian-quandl <no ingestions>
us_equities 2025-03-21 14:56:38.203846
```